# Introduction

Notice a wrong prediction in the test set, all the result is negative LOL

In [1]:
import pandas as pd
import numpy as np
import calendar
pd.set_option('display.max_columns', 100)

## Function

### Preprocessing Data

In [2]:
def X_preprocessing(df):
    
    # checking resposne variable
    if 'SalePrice' not in df:
        column_interval_dup = column_interval.copy()
        column_interval_dup.remove('SalePrice')
        
    # drop useless column "id"
    df = df.drop('Id', axis=1)
    
    df_nominal = df[column_nominal].copy()
    df_interval = df[column_interval_dup].copy()

    # preprocessing on interval dataframe
    ## simple null value imputation: interval to 0
    ## concat to the actual table
    df_interval = df_interval.fillna(0)
    df = df.drop(column_interval_dup, axis=1)
    df = pd.concat([df, df_interval], axis=1)

    # binary data convesion
    df['CentralAir'] = df[column_binary]['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)

    # preprocessing on nonimal dataframe
    ## simple null value imputation: categorical to NaN
    df_nominal = df_nominal.drop(col_major_missing, axis = 1)
    df_nominal = df_nominal.fillna(nominal_mode)
    

    ## convert MSSubClass value to string
    df_nominal['MSSubClass'] = df_nominal['MSSubClass'].astype('string')
    df_nominal['MSSubClass'] = df_nominal['MSSubClass'].apply(lambda x: 'MSSC_'+x)

    ## all set, convert all the categorical variable to dummies variables
    df_nominal = pd.get_dummies(df_nominal)

    ## concat to the actual table
    df = df.drop(column_nominal, axis=1)
    df = pd.concat([df, df_nominal], axis=1)

    # preprocessing on time dataframe
    ## time approach-renovation: comparing to the year built
    ## time approach-soldyear: comparing to the year built
    df['YearRemodAdd'] = df[column_time]['YearRemodAdd'] - df[column_time]['YearBuilt'] 
    df['YrSold'] = df[column_time]['YrSold'] - df[column_time]['YearBuilt']

    ## time approach-built: comparing to the oldest listing on the market
    df_yrbuilt_min = df[column_time]['YearBuilt'].min()
    df['YearBuilt'] = df[column_time]['YearBuilt'].apply(lambda x: x - df_yrbuilt_min)

    ## time apprach-soldmonth: to dummies variable
    df['MoSold'] = df['MoSold'].apply(lambda x: 'sold_'+calendar.month_name[x])
    df_dum_months = pd.get_dummies(df['MoSold'])
    df = df.drop(['MoSold'], axis=1)

    df = pd.concat([df,df_dum_months], axis=1)

    if 'SalePrice' in df:
    # final to split between dependent variable to resposne
        X = df.drop('SalePrice', axis=1)
        y = df['SalePrice']

    else:
        print('This is test dataset return only X...')
        X = df.copy()
        y = None

    return X, y

## Import data(s)

In [3]:
train_df = pd.read_csv('data/train.csv')
df_cols_process = pd.read_csv('data/column_types_and_process_imputation.csv')

## Data Exploration

-> Numerical - Categorical (two general groups) <- 

- Useless (unique identifier usually useless)
- Nominal (as named or categorical)
- Binary (either 1/0)
- Ordinal (Ordinal)
- Count (Integer number starting from 0 exclude negative number)
- Time (consiting seasonal and date)
- Interval (Examples include percentages, temperatures, and income.)

In [4]:
df_cols_process['data types'] = df_cols_process['data types'].apply(lambda x: x.strip())

In [5]:
df_cols_process['data types'].unique()

array(['useless', 'nominal', 'interval', 'ordinal', 'time', 'binary'],
      dtype=object)

In [6]:
column_interval = list(df_cols_process[df_cols_process['data types'] == 'interval']['column']) # no need processing
column_nominal = list(df_cols_process[df_cols_process['data types'] == 'nominal']['column']) 
column_ordinal = list(df_cols_process[df_cols_process['data types'] == 'ordinal']['column']) # no need processing
column_binary = list(df_cols_process[df_cols_process['data types'] == 'binary']['column'])
column_time = list(df_cols_process[df_cols_process['data types'] == 'time']['column'])

## Data Pre-processing

### Drop useless data types

In [8]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# from sklearn.preprocessing import OrdinalEncoder

In [7]:
train_df_temp = train_df.drop('Id',axis = 1).copy()

In [8]:
train_df_nominal = train_df_temp[column_nominal].copy()
train_df_interval = train_df_temp[column_interval].copy()

### Data Imputation

#### Nominal

In [9]:
# build an array for removing nominal columns that have missinv value more than 30%.
col_major_missing = np.array(train_df_nominal.columns[train_df_nominal.isnull().sum()/train_df_nominal.shape[0] > 0.30])

# drop it
train_df_nominal = train_df_nominal.drop(col_major_missing, axis=1)

# assign mode to a variable "nominal mode".
nominal_mode = train_df_nominal.mode().iloc[0]

# simple null value imputation: Mode
train_df_nominal = train_df_nominal.fillna(nominal_mode)

#### Interval 

In [10]:
# simple null value imputation: interval to 0
train_df_interval = train_df_interval.fillna(0)

In [11]:
train_df_temp = train_df_temp.drop(column_interval, axis=1)
train_df_temp = pd.concat([train_df_temp, train_df_interval], axis=1)

### Nominal data to Dummies

In [12]:
train_df_nominal['MSSubClass'] = train_df_nominal['MSSubClass'].astype('string')
train_df_nominal['MSSubClass'] = train_df_nominal['MSSubClass'].apply(lambda x: 'MSSC_'+x)

In [13]:
# from 43 to 281 columns
train_df_nominal = pd.get_dummies(train_df_nominal)

In [14]:
train_df_temp = train_df_temp.drop(column_nominal, axis=1)
train_df_temp = pd.concat([train_df_temp, train_df_nominal], axis=1)

### Change the value of binary data

In [15]:
train_df_temp['CentralAir'] = train_df_temp[column_binary]['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)

### Change the value of time data

In [16]:
# unique approach on how to process the time data
train_df_temp['YearRemodAdd'] = train_df_temp[column_time]['YearRemodAdd'] - train_df_temp[column_time]['YearBuilt']
train_df_temp['YrSold'] = train_df_temp[column_time]['YrSold'] - train_df_temp[column_time]['YearBuilt']

train_df_temp_yrbuilt_min = train_df_temp[column_time]['YearBuilt'].min()
train_df_temp['YearBuilt'] = train_df_temp[column_time]['YearBuilt'].apply(lambda x: x - train_df_temp_yrbuilt_min)

In [17]:
# turn YrSold compare to Yr Built.
train_df_temp['MoSold'] = train_df_temp['MoSold'].apply(lambda x: 'sold_'+calendar.month_name[x])
train_df_temp_dum_months = pd.get_dummies(train_df_temp['MoSold'])
train_df_temp = train_df_temp.drop(['MoSold'], axis=1)

train_df_temp = pd.concat([train_df_temp,train_df_temp_dum_months], axis=1)

## Define Dependent and Response Data of Training

In [18]:
X_train = train_df_temp.drop('SalePrice', axis=1)
Y_train = train_df_temp['SalePrice']

---

## Feature Scalling

### Z-score

In [19]:
# Zscore using SKLearn

from sklearn.preprocessing import scale
# scale(X_orig, axis=0, with_mean=True, with_std=True, copy=True)

def zscore_normalize_features(X, mu = False, sigma = False):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    
    if (mu is False) and (sigma is False):
        # find the mean of each column/feature
        mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
        # find the standard deviation of each column/feature
        sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
        # element-wise, subtract mu for that column from each example, divide by std for that column
        
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

In [20]:
# using SKLearn
# X_norm = scale(X_train, with_mean = True, with_std = True, copy = True)
X_train_scale, mu, sigma = zscore_normalize_features(X_train)

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_scale = scaler.transform(X_train)
X_train_scale = pd.DataFrame(X_train_scale, columns = X_train.columns)

## Fitting Linear Regression

In [22]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [23]:
X_train_arr = np.array(X_train_scale)
linear_model.fit(X_train_arr, Y_train)

LinearRegression()

In [24]:
Y_train_pred = linear_model.predict(X_train_arr)

In [51]:
from sklearn.ensemble import RandomForestRegressor

In [53]:
random_forest = RandomForestRegressor()
random_forest.fit(X_train_arr, Y_train)

RandomForestRegressor()

In [54]:
Y_train_pred = random_forest.predict(X_train_arr)

## Root Mean Squared Error

In [25]:
from sklearn.metrics import mean_squared_error
import math

In [26]:
train_eval_metrics = math.sqrt(mean_squared_error(Y_train_pred, Y_train))

In [27]:
train_eval_metrics

20809.892056569515

In [55]:
train_eval_metrics = math.sqrt(mean_squared_error(Y_train_pred, Y_train))

In [56]:
train_eval_metrics

10504.458338273136

## Test Dataset

In [61]:
test_df = pd.read_csv('data/test.csv')

In [29]:
X_test, y_test = X_preprocessing(test_df)

This is test dataset return only X...


## Adjust the column of train and test datasource

In [30]:
X_train_scale_final, X_test = X_train_scale.align(X_test, join = 'left', axis=1)

### Using z-scaler to transform it

In [31]:
X_test_scale = scaler.transform(X_test)
X_test_scale_final = pd.DataFrame(X_test_scale, columns= X_test.columns)

X_test_scale_final = X_test_scale_final.fillna(0)

In [49]:
X_test_scale_final

,OverallQual,OverallCond,YearBuilt,YearRemodAdd,CentralAir,YrSold,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_MSSC_120,MSSubClass_MSSC_160,MSSubClass_MSSC_180,MSSubClass_MSSC_190,MSSubClass_MSSC_20,MSSubClass_MSSC_30,MSSubClass_MSSC_40,MSSubClass_MSSC_45,MSSubClass_MSSC_50,MSSubClass_MSSC_60,MSSubClass_MSSC_70,MSSubClass_MSSC_75,MSSubClass_MSSC_80,MSSubClass_MSSC_85,MSSubClass_MSSC_90,...,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,sold_April,sold_August,sold_December,sold_February,sold_January,sold_July,sold_June,sold_March,sold_May,sold_November,sold_October,sold_September
0,-0.795151,0.381743,-0.571923,-0.555742,0.263813,0.411777,0.645747,0.110763,-0.570750,0.053428,0.604293,-0.672923,-0.400017,-0.689929,-0.795163,-0.120242,-1.179256,-0.819964,-0.241061,-1.026041,-0.761621,-1.062465,-0.211454,-0.934130,-0.951226,0.203422,-1.026858,1.202536,0.365179,-0.704483,-0.359325,-0.116339,1.882709,-0.068692,-0.087688,-0.251724,-0.212360,-0.083045,-0.144841,1.312966,-0.222721,-0.052414,-0.091035,-0.330791,-0.507481,-0.20702,-0.105263,-0.203395,-0.117851,-0.192177,...,0.271163,-0.064238,0.731591,-0.114827,-0.253259,-0.078757,-0.600559,-0.563640,-0.637614,1.062205,0.0,-0.184376,-0.098397,-0.045376,0.221022,-0.037037,-0.156721,-0.078757,-0.069409,0.194085,-0.256307,-0.144841,0.299253,-0.1742,-0.052414,-0.037037,-0.078757,-0.058621,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,-0.326954,-0.301962,-0.205214,-0.192177,-0.203395,-0.436881,2.184205,-0.279797,-0.403014,-0.239176,-0.254786,-0.212360
1,-0.071836,0.381743,-0.671285,-0.555742,0.263813,0.510984,0.674605,0.375850,0.027027,1.051363,-0.288653,-0.365032,0.619239,0.430511,-0.795163,-0.120242,-0.354966,-0.819964,-0.241061,-1.026041,1.227585,0.163779,-0.211454,-0.318683,-0.951226,0.196807,-1.026858,-0.753188,2.384400,-0.160950,-0.359325,-0.116339,-0.270208,-0.068692,25.116309,-0.251724,-0.212360,-0.083045,-0.144841,1.312966,-0.222721,-0.052414,-0.091035,-0.330791,-0.507481,-0.20702,-0.105263,-0.203395,-0.117851,-0.192177,...,0.271163,-0.064238,0.731591,-0.114827,-0.253259,-0.078757,-0.600559,-0.563640,-0.637614,1.062205,0.0,-0.184376,-0.098397,-0.045376,0.221022,-0.037037,-0.156721,-0.078757,-0.069409,0.194085,-0.256307,-0.144841,0.299253,-0.1742,-0.052414,-0.037037,-0.078757,-0.058621,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995,-0.326954,-0.301962,-0.205214,-0.192177,-0.203395,-0.436881,2.184205,-0.279797,-0.403014,-0.239176,-0.254786,-0.212360
2,-0.795151,-0.517200,0.620424,-0.514873,0.263813,-0.778707,0.472599,0.332053,-0.570750,0.761852,-0.288653,-0.974021,-0.295127,-0.607125,0.811239,-0.120242,0.216136,-0.819964,-0.241061,0.789741,1.227585,0.163779,-0.211454,-0.318683,0.600495,0.282797,0.311725,0.042202,0.939819,-0.191147,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.251724,-0.212360,-0.083045,-0.144841,-0.761634,-0.222721,-0.052414,-0.091035,-0.330791,1.970518,-0.20702,-0.105263,-0.203395,-0.117851,-0.192177,...,0.271163,-0.064238,0.731591,-0.114827,-0.253259,-0.078757,-0.600559,1.774184,-0.637614,-0.941438,0.0,-0.184376,-0.098397,-0.04537

---

In [38]:
Y_test_pred = linear_model.predict(np.array(X_test_scale_final))

In [57]:
Y_test_pred = random_forest.predict(np.array(X_test_scale_final))

range(1461, 2919)

In [75]:
df_submission

,SalePrice
Id,
1461,130219.50
1462,153467.72
1463,178361.56
1464,184742.50
1465,198355.42
...,...
2915,84317.00
2916,86261.00
2917,149045.50


In [68]:
df_submission = pd.DataFrame(Y_test_pred, index=range(1461, 2920))
df_submission.columns = ['SalePrice']
df_submission.index.name = 'Id'

In [74]:
df_submission.to_csv('submission_15_january_2023.csv')

In [26]:
train_eval_metrics = math.sqrt(mean_squared_error(Y_train_pred, Y_train))

In [27]:
train_eval_metrics

20809.892056569515

# Resources

https://towardsdatascience.com/7-data-types-a-better-way-to-think-about-data-types-for-machine-learning-939fae99a689

https://medium.com/analytics-vidhya/ways-to-handle-categorical-column-missing-data-its-implementations-15dc4a56893#:~:text=Step%201%3A%20Find%20which%20category,and%20keep%20newly%20imputed%20columns.&text=Advantage%3A%20Simple%20and%20easy%20to%20implement%20for%20categorical%20variables%2Fcolumns.



https://stackoverflow.com/questions/62653114/how-can-i-deal-with-this-git-warning-pulling-without-specifying-how-to-reconci

# Archive

## Linear Regression Simplification

In [151]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [163]:
X = fetch_california_housing()['data']
X = pd.DataFrame(X, columns=fetch_california_housing()['feature_names'])
Y = fetch_california_housing()['target']
Y = pd.DataFrame(Y, columns = fetch_california_housing()['target_names'])

In [153]:
X_train, X_test = train_test_split(X, test_size=0.2)

Y_train = Y.iloc[X_train.index]
Y_test = Y.drop(X_train.index)

In [190]:
def zscore_normalize_features(X, mu = False, sigma = False):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    
    if (mu is False) and (sigma is False):
        # find the mean of each column/feature
        mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
        # find the standard deviation of each column/feature
        sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
        # element-wise, subtract mu for that column from each example, divide by std for that column
        
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

In [211]:
X_train_scale, mu, sigma = zscore_normalize_features(X_train, mu = False, sigma = False)

In [212]:
X_test_scale, mu_2, sigma_2 = zscore_normalize_features(X_test, mu = mu, sigma = sigma)

In [203]:
linear_model = LinearRegression() 
linear_model.fit(X_train_scale, Y_train)

LinearRegression()

In [214]:
Y_train_pred = linear_model.predict(X_train_scale)

In [215]:
Y_test_pred = linear_model.predict(X_test_scale)

In [216]:
math.sqrt(mean_squared_error(Y_train_pred, Y_train))

0.7215088154647735

In [217]:
math.sqrt(mean_squared_error(Y_test_pred, Y_test))

1.4719059330502595